### Conver Format to yolo
현재 어노테이션 정보들은 모두 절대좌표로 찍혀 있는 상태이다. 이를 정규화된 상대 좌표로 변환한다.

### Yolo Annotation format
yolo dataset의 경우 이미지 하나에 annotation 정보와 클래스 넘버를 포함하고 있는 txt파일 하나가 매핑되는게 일반적이며, txt파일은 아래와 같이 구성된다.

```
클래스 번호   x좌표(어노테이션 박스 중심점)   y좌표(어노테이션 박스 중심점)    annotation box's width    annotation box's height
```

### AI Hub's annotation file


In [15]:
import os
import json
import shutil
from glob import glob

labels_dir = "./new_dataset/labels"  # yolo format으로 저장될 위치
imgs_dir = "./new_dataset/images"
cnt = {0:0, 5:0, 6:0}
class_num = {0:0, 5:1, 6:2}  # 0 : 정상(0번) 5 : 검은 썩음병(1번) 6 : 배추노균병(2번)

image_filenames = [i.replace("\\", "/") for i in glob(r"D:\origin_dataset\images\*\*")]
json_filenames = [i.replace("\\", "/") for i in glob(r"D:\origin_dataset\labels\*\*.json")]
for json_path, image_path in zip(json_filenames, image_filenames):
    with open(json_path, "r", encoding="utf8") as file:
        json_file = json.load(file)

    image_filename = json_file["description"]["image"]
    img_extension = image_filename[image_filename.rfind(".") : ]
    image_width = json_file['description']['width']
    image_height = json_file['description']['height']

    cnt[json_file["annotations"]['disease']] += 1

    if cnt[json_file["annotations"]['disease']] <= 592:
        bboxes = ""  # 바운딩 박스 좌표를 담을 빈 문자열
        for point in json_file["annotations"]["points"]:  # 모든 바운딩 박스 좌표를 순회하면서
            x_tl, y_tl = point["xtl"], point["ytl"]  # x,y Top-left 좌표
            x_br, y_br = point["xbr"], point["ybr"]  # x,y Bottom-right 좌표

            bounding_box_width = (x_br - x_tl) / image_width  # 바운딩 박스 가로 길이 정규화
            bounding_box_height = (y_br - y_tl) / image_height  # 바운딩 박스 세로 길이 정규화
            center_x = int((x_tl + x_br) / 2)  # 바운딩 박스 중심점의 x좌표
            center_y = int((y_tl + y_br) / 2)  # 바운딩 박스 중심점의 y좌표

            normed_center_x = center_x/image_width  # 정규화된 바운딩 박스 중심점의 x좌표
            normed_center_y = center_y/image_height  # 정규화된 바운딩 박스 중심점의 y좌표
            bbox = f"{class_num[json_file['annotations']['disease']]} {normed_center_x} {normed_center_y} {bounding_box_width} {bounding_box_height}\n"
            bboxes += bbox

        txt_filename = os.path.join(labels_dir, image_filename)
        with open(txt_filename.replace(img_extension, ".txt"), "w", encoding="utf8") as txt:
            txt.write(bboxes)
        shutil.move(image_path, os.path.join(imgs_dir, image_filename))

    file.close()
    txt.close()

In [5]:
from glob import glob
import json

json_filenames = [i.replace("\\", "/") for i in glob(r"D:\origin_dataset\labels\*\*.json")]

total_width = 0
total_height = 0
for json_path in json_filenames:
    with open(json_path, "r", encoding="utf8") as file:
        json_file = json.load(file)
    image_width = json_file['description']['width']
    image_height = json_file['description']['height']

    total_width += image_width
    total_height += image_height

    file.close()

print(f"이미지 평균 가로 길이 : {int(total_width / len(json_filenames))}px")
print(f"이미지 평균 세로 길이 : {int(total_height / len(json_filenames))}px")

이미지 평균 가로 길이 : 3679px
이미지 평균 세로 길이 : 3233px


In [10]:
from glob import glob
import json

json_filenames = [i.replace("\\", "/") for i in glob(r"./detection_dataset/*/labels/*/*.json")]

cnt = {0:0, 5:0, 6:0}
for json_path in json_filenames:
    with open(json_path, "r", encoding="utf8") as file:
        json_file = json.load(file)
    if len(json_file["annotations"]["points"]) >= 1:
        cnt[json_file["annotations"]["disease"]] += 1
    else:
        print(f"thers's no any points. filename is {json_file['description']['image']}")
    file.close()

In [11]:
cnt

{0: 1183, 5: 1760, 6: 592}

In [16]:
import splitfolders

splitfolders.ratio("./new_dataset", "./new_detection", seed=11, ratio=(0.6, 0.2, 0.2))

Copying files: 3552 files [02:06, 28.00 files/s] 


### V8 training params

|params|default|설명|
|---|---|---|
|model|None|사전에 학습된 모델의 가중치를 사용하거나, 모델 자체를 사용하기 위한 parameter|
|data|None|데이터셋 명세가 적힌 yaml 파일의 경로|
|epochs|100|훈련 횟수|
|time|None|학습 시간에 대한 limit을 설정한다.|
|patience|100|early stop을 위한 매개변수. 해당 횟수만큼 모델에 개선이 없으면 조기종료한다.|
|imgsz|640|이미지의 가로 세로 크기를 조절한다. 학습 정확도와 계산 복잡성에 영향을 직접적으로 미친다.|
|save|True|학습 결과를 best/last.pt로 저장한다.|
|save_preiod|-1|학습 중 checkpoint를 저장할지 말지에 대한 여부. -1은 비활성화를 의미하며, 지정하는 숫자의 epochs마다 저장한다.|
|cache|None|데이터셋 이미지를 메모리 혹은 디스크에 캐싱하게 하여 메모리 사용량은 증가하지만 교육 속도는 향상된다.|

In [17]:
import os
from ultralytics import YOLO

os.environ['WANDB_DISABLED'] = 'true'

def main():
    model = YOLO("./detection_model/yolov8l.pt", task="detect")
    model.train(
        model = "./detection_model/yolov8l.pt",
        data = "./new_detection/cabbage_data.yaml",
        imgsz=640,
        epochs=25,
        batch=16,  # batch=-1 -> 장비가 허용하는 범위 내에서 동적으로 설정
        project="detection_model",
        name="cabbage_detection",
        workers=2,
        plots=True,
        device=0,  # gpu 사용
        patience=10,
        translate=0.0,
    )

if __name__ == "__main__":
    main()

New https://pypi.org/project/ultralytics/8.2.2 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.9.12 torch-2.1.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: task=detect, mode=train, model=./detection_model/yolov8l.pt, data=./new_detection/cabbage_data.yaml, epochs=25, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=detection_model, name=cabbage_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=Fa

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
WARNING  NMS time limit 0.550s exceeded
AMP: checks passed 
train: Scanning D:\CudaTest\cabbage\new_detection\train\labels...:   0%|          | 0/1065 [00:00<?, ?it/s]Scanning D:\CudaTest\cabbage\new_detection\train\labels... 26 images, 0 backgrounds, 0 corrupt:   2%|▏         | 26/1065 [00:00<00:04, 248.79it/s]Scanning D:\CudaTest\cabbage\new_detection\train\labels... 106 images, 0 backgrounds, 0 corrupt:  10%|▉         | 106/1065 [00:00<00:01, 547.88it/s]Scanning D:\CudaTest\cabbage\new_detection\train\labels... 205 images, 0 backgrounds, 0 corrupt:  19%|█▉        | 205/1065 [00:00<00:01, 702.09it/s]Scanning D:\CudaTest\cabbage\new_detection\train\labels... 293 images, 0 backgrounds, 0 corrupt:  28%|██▊       | 293/1065 [00:00<00:01, 752.89it/s]Scanning D:\CudaTest\cabbage\new_detection\train\labels... 393 images, 0 backgrounds, 0 corrupt:  37%|███▋      | 393/1065 [00:00<00:

lr/pg0,▃▆███▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁
lr/pg1,▃▆███▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁
lr/pg2,▃▆███▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁
metrics/mAP50(B),▁▁▂▃▅▃▇▇▇▇▇▇▇████████████
metrics/mAP50-95(B),▁▁▁▂▄▂▅▆▆▆▇▆▆▇▇▇▇▇▇▇█████
metrics/precision(B),▁▁▂▃▆▃▇▇▇▇▇▇▇█▇█▇████████
metrics/recall(B),▄▁▁▃▅▂▆▇▆▇▇▇▇▇▇▇▇██▇█████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▆██▇▇▆▆▅▅▅▅▅▄▄▄▃▃▃▃▂▂▂▁▁▁


In [20]:
custom_model = YOLO(r"C:\Users\admin\Desktop\CudaTest\runs\detect\cabbage3\weights\best.pt")
results = custom_model.predict(source=r"C:\Users\admin\Desktop\testing\*", save=True)


image 1/12 C:\Users\admin\Desktop\testing\V006_79_0_00_03_03_12_0_b06_20201020_0005_S01_1.jpg: 320x640 1 normal, 28.4ms
image 2/12 C:\Users\admin\Desktop\testing\V006_79_0_00_03_03_12_0_b06_20201020_0070_S01_1.jpg: 320x640 1 normal, 15.7ms
image 3/12 C:\Users\admin\Desktop\testing\V006_79_0_00_03_03_12_0_b06_20201020_0072_S01_1.jpg: 320x640 1 normal, 31.9ms
image 4/12 C:\Users\admin\Desktop\testing\V006_79_0_00_03_03_12_0_b06_20201020_0092_S01_1.jpg: 320x640 1 normal, 30.6ms
image 5/12 C:\Users\admin\Desktop\testing\V006_79_1_05_03_03_12_1_5978e_20201029_12_a0002.jpeg: 640x640 1 black_rot, 51.7ms
image 6/12 C:\Users\admin\Desktop\testing\V006_79_1_05_03_03_12_1_5978e_20201029_12_a0005.jpeg: 640x640 1 black_rot, 62.0ms
image 7/12 C:\Users\admin\Desktop\testing\V006_79_1_05_03_03_12_1_5978e_20201029_12_a0007.jpeg: 640x640 1 black_rot, 81.5ms
image 8/12 C:\Users\admin\Desktop\testing\V006_79_1_05_03_03_12_1_5978e_20201029_12_b0004.jpeg: 640x640 1 black_rot, 73.1ms
image 9/12 C:\Users\adm

In [ ]:
import numpy as np

for result in results:
    uniq, cnt = np.unique(result.boxes.cls.cpu().numpy(), return_counts=True)
    uniq_cnt_dict = dict(zip(uniq, cnt))

    print()